In [3]:
import os
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from tkinter import Tk, filedialog
import json


def ler_email_excel():
    root = Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename()

    df = pd.read_excel(file_path)
    return df, file_path  # Retorna também o caminho do arquivo


def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return HTTPBasicAuth(username, password)


def obter_ids_api(email_name, auth):
    base_url = "https://secure.p04.eloqua.com/api/REST/2.0/assets/emails"
    search_url = f"{base_url}?search=name='{email_name}'&depth=partial"

    response = requests.get(search_url, auth=auth)
    data = response.json()

    if 'elements' in data and data['elements']:
        email_info = data['elements'][0]
        email_footer_id = email_info.get('emailFooterId', None)
        email_header_id = email_info.get('emailHeaderId', None)
        email_api_id = email_info.get('id', None)
        template_id = email_info.get('sourceTemplateId', None)

        return email_footer_id, email_header_id, email_api_id, template_id
    else:
        return None, None, None, None


def obter_nome_por_id(api_url, email_id, auth):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/assets/email/{api_url}/{email_id}"
    response = requests.get(url, auth=auth)

    if response.status_code == 200:
        try:
            data = response.json()
            if 'name' in data:
                return data['name']
            else:
                return None
        except json.decoder.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON: {e}")
            return None
    else:
        # print(f"Erro na solicitação: {response.status_code} - {response.text}")
        return None


def obter_nome_template_por_id(template_id, auth):
    url = f"https://secure.p04.eloqua.com/api/REST/2.0/assets/email/template/{template_id}"
    response = requests.get(url, auth=auth)

    if response.status_code == 200:
        try:
            data = response.json()
            if 'name' in data:
                return data['name']
            else:
                return None
        except json.decoder.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON: {e}")
            return None
    else:
        print(f"Erro na solicitação: {response.status_code} - Template Not Found")
        return None


def atualizar_planilha(df, email_name, email_footer_id, email_header_id, email_api_id, template_id, auth):
    mask = df['Email Name'] == email_name
    # Convertendo para float, ajuste o tipo conforme necessário
    df.loc[mask, 'Footer ID'] = pd.to_numeric(email_footer_id, errors='coerce')
    df.loc[mask, 'Header ID'] = pd.to_numeric(email_header_id, errors='coerce')
    df.loc[mask, 'Email ID'] = pd.to_numeric(email_api_id, errors='coerce')
    df.loc[mask, 'Template ID'] = pd.to_numeric(template_id, errors='coerce')

    # Após obter os IDs, obter os nomes associados
    email_footer_name = obter_nome_por_id('footer', email_footer_id, auth)
    email_header_name = obter_nome_por_id('header', email_header_id, auth)
    template_name = obter_nome_template_por_id(template_id, auth)

    # Atualizar a planilha com os nomes obtidos
    df.loc[mask, 'Footer Name'] = email_footer_name
    df.loc[mask, 'Header Name'] = email_header_name
    df.loc[mask, 'Template Name'] = template_name


auth = obter_autenticacao()
df_emails, file_path = ler_email_excel()

for index, row in df_emails.iterrows():
    email_name = row['Email Name']

    if pd.notna(email_name):
        email_footer_id, email_header_id, email_api_id, template_id = obter_ids_api(
            email_name, auth)

        if email_footer_id is not None or email_header_id is not None or template_id is not None:
            atualizar_planilha(df_emails, email_name, email_footer_id,
                               email_header_id, email_api_id, template_id, auth)
            print(f"Encontrado e atualizado planilha '{email_name}'.")
        else:
            print(
                f"Não foi possível encontrar informações para o e-mail '{email_name}'.")


# Salvar a planilha atualizada
df_emails.to_excel(file_path, index=False)
print("Planilha atualizada com sucesso. Certifique-se de fechar o arquivo Excel antes de executar qualquer ação adicional.")

Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'TAX PROF BR EVENTO ECOSSISTEMA DA CONTABILIDADE DIGITAL - OBRIGADO PELA INSCRICAO'.


C:\Users\6125974\AppData\Local\Temp\ipykernel_10840\3458336481.py:96: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Latam Portuguese' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Footer Name'] = email_footer_name
C:\Users\6125974\AppData\Local\Temp\ipykernel_10840\3458336481.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Latam Portuguese' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Header Name'] = email_header_name


Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'br_tax_nsl_2022318_news_contabil'.
Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'TAX PROF BR CAMPANHA INTEGRACAO COM ERPs - CONTADOR 1'.
Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'TAX PROF BR CAMPANHA INTEGRACAO COM ERPs - DESENVOLVEDOR 1'.
Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'TAX PROF BR CAMPANHA INTEGRACAO COM ERPs - DESENVOLVEDOR 2'.
Erro na sol

C:\Users\6125974\AppData\Local\Temp\ipykernel_10840\3458336481.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Julia Email Media Code 6 - Hive9 Testing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Template Name'] = template_name


Encontrado e atualizado planilha 'br_tax_emkt_PCMC_mar_30'.
Encontrado e atualizado planilha 'br_tax_emkt_PCMC_base_dominioweb'.
Encontrado e atualizado planilha 'br_tax_emkt_Flasback_2023_ultimashoras_feb_28-'.
Encontrado e atualizado planilha 'br_tax_emkt_black22_lançamento_módulos'.
Encontrado e atualizado planilha 'br_tax_fm_202385_emkt_conexao_contabil_2023_Foz do Iguaçú'.
Encontrado e atualizado planilha 'br_tax_emkt_digital_banking_ative_sua_conta'.
Encontrado e atualizado planilha 'br_tax_emkt_digital_banking_atualize_para_ acessar_novas_funcionalidades'.
Encontrado e atualizado planilha 'br_tax_emkt_comunicado_TAKEOFF2_2023'.
Encontrado e atualizado planilha 'br_tax_emkt_comunicado_CON23_Participamentes_SYNERGY'.
Encontrado e atualizado planilha 'br_tax_emkt_comunicado_MiniBA_3'.
Erro na solicitação: 400 - [{"type":"EndpointParameterError","parameter":"id","requirement":{"type":"IdRequirement","isRelativeAllowed":false},"value":"None"}]
Encontrado e atualizado planilha 'br_tax